# Crashes on a Street

In [1]:
import json
import pytz
import folium
from folium.features import DivIcon
from folium.plugins import HeatMap

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from matplotlib._color_data import TABLEAU_COLORS

from crash_data_analysis import CrashDataAnalysis

In [2]:
cda = CrashDataAnalysis()

## Find ID of street

In [3]:
query = """
select
sr.gid
, sr.lrsroute
, st_AsGeoJSON(sr.geom) as street_line
, sc.fullname
, count(distinct sc.masterid)
, row_number() over (partition by sr.gid, sr.lrsroute order by count(distinct sc.masterid) desc) 
    as fullname_priority

from street_routes sr
inner join street_centerline sc using (lrsroute)

group by 1,2,3,4
"""

street_ids = pd.read_sql(query, cda.conn)

In [21]:
streets_to_map = street_ids[street_ids.lrsroute.str.contains('SPEER')].reset_index()
streets_to_map

,index,gid,lrsroute,street_line,fullname,count,fullname_priority
0,1489,1452,ESPEERN BD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E SPEER BLVD,15,1
1,1490,1453,ESPEERS BD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E SPEER BLVD,13,1
2,3803,3707,ESPEERSWBBD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",E SPEER BLVD,2,1
3,3997,3899,SPEER BD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",N SPEER BLVD,12,1
4,3998,3900,SPEERN BD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",N SPEER BLVD,36,1
5,3999,3901,SPEERS BD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",N SPEER BLVD,31,1
6,4000,3902,SPEERSNB BD,"{""type"":""MultiLineString"",""coordinates"":[[[-10...",N SPEER BLVD,2,1


In [22]:
this_map = folium.Map(prefer_canvas=True, tiles='Stamen Toner')

street_color = TABLEAU_COLORS['tab:orange']

for idx, row in streets_to_map.iterrows():
    folium.GeoJson(row.street_line, style_function=lambda x: {'color': street_color}).add_to(this_map)

    street_line_dict = json.loads(row.street_line)
    middle_point = int(np.round(len(street_line_dict['coordinates'][0]) / 2))
    
    folium.map.Marker(
        [street_line_dict['coordinates'][0][middle_point][1], street_line_dict['coordinates'][0][middle_point][0]]
        , icon=DivIcon(
            icon_size=(250,36)
            , icon_anchor=(0,0)
            , html=f'<div style="font-size: 20pt; color:{street_color}">gid: {row.gid}</div>'
        )
     ).add_to(this_map)

this_map.fit_bounds(this_map.get_bounds())
this_map

In [16]:
street_gid = 132

In [17]:
start_date = pytz.timezone('America/Denver').localize(datetime(2013,1,1))
end_date = pytz.timezone('America/Denver').localize(datetime(2023,5,1))
most_recent_crash = cda.most_recent_crash_timestamp()

In [18]:
query = f"""
select
sr.gid
, sr.lrsroute
, st_length(sr.geom_denver) / 5280 as length_miles
, c.sbi
, c.fatality
, c.pedestrian_ind
, c.bicycle_ind
, c.geo_lon
, c.geo_lat
, c.reported_date
, c.reported_date at time zone 'America/Denver' as reported_date_local
, c.incident_address_corrected

from street_routes sr
inner join crashes c on st_dwithin(sr.geom_denver, c.geom_denver, 25)

where sr.gid = {street_gid}
and reported_date at time zone 'America/Denver' > '{start_date.strftime('%Y-%m-%d')}'
and reported_date at time zone 'America/Denver' < '{end_date.strftime('%Y-%m-%d')}'
and not at_freeway

order by reported_date desc
"""

street_crashes = pd.read_sql(query, cda.conn)

In [19]:
street_crashes.groupby(street_crashes.reported_date_local.dt.year).agg(
    crashes=('reported_date_local', 'count')
    , fatalities=('fatality', 'sum')
    , sbi=('sbi', 'sum')
    , pedestrian_ind=('pedestrian_ind', 'sum')
    , bicycle_ind=('bicycle_ind', 'sum')
).astype(int)

,crashes,fatalities,sbi,pedestrian_ind,bicycle_ind
reported_date_local,,,,,
2013,117,1,1,2,6
2014,129,0,1,7,2
2015,143,0,2,3,4
2016,155,0,1,3,5
2017,153,0,1,5,3
2018,162,0,1,6,6
2019,174,1,3,8,5
2020,91,0,1,1,3
2021,83,0,1,3,2


In [20]:
this_map = folium.Map(prefer_canvas=True)
HeatMap(street_crashes[['geo_lat', 'geo_lon']]).add_to(this_map)
this_map.fit_bounds(this_map.get_bounds())
this_map

In [11]:
days_in_data = (end_date - start_date).days

In [12]:
days_between_crashes = days_in_data / len(street_crashes)
days_between_crashes

8.531674208144796

In [13]:
crashes_per_mile_per_week = (len(street_crashes) / street_crashes.iloc[0].length_miles) / (days_in_data/7)
crashes_per_mile_per_week

0.5105641032769805

In [14]:
street_crashes.sort_values(by='geo_lat')

,gid,lrsroute,length_miles,sbi,fatality,pedestrian_ind,bicycle_ind,geo_lon,geo_lat,reported_date,reported_date_local,incident_address_corrected
441,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.989829,39.756451,2013-01-27 04:42:00+00:00,2013-01-26 21:42:00,PARK AVE W / MARKET ST
405,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.989829,39.756451,2013-12-29 21:30:00+00:00,2013-12-29 14:30:00,PARK AVE W / WALNUT ST
397,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.989829,39.756451,2014-02-17 23:28:00+00:00,2014-02-17 16:28:00,PARK AVE W / MARKET ST
374,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.989829,39.756451,2015-03-01 18:08:00+00:00,2015-03-01 11:08:00,WALNUT ST / PARK AVE W
371,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.989829,39.756451,2015-04-20 07:04:00+00:00,2015-04-20 01:04:00,PARK AVE W / MARKET ST
...,...,...,...,...,...,...,...,...,...,...,...,...
290,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.968477,39.772910,2017-03-17 20:50:00+00:00,2017-03-17 14:50:00,E 40TH AVE / N FRANKLIN ST
30,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.968477,39.772910,2022-05-25 22:17:00+00:00,2022-05-25 16:17:00,N FRANKLIN ST / E 40TH AVE
331,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.968477,39.772910,2016-05-28 23:05:00+00:00,2016-05-28 17:05:00,WALNUT ST / E 40TH AVE
314,3628,WALNUT ST1,1.606991,False,False,0.0,0.0,-104.968477,39.772910,2016-09-27 17:03:00+00:00,2016-09-27 11:03:00,E 40TH AVE / N FRANKLIN ST


In [15]:
cda.conn.dispose()